In [1]:
###Importation des packages

import numpy as np
import time

In [2]:
def generation(n, alpha=1.7, beta=0.9, gamma=10, delta=10):
    # Initialize samples array with zeros
    sample = np.zeros(n)
    
    # Constants that do not depend on the sample index and thus can be computed once
    if alpha != 1:
        S_alpha_beta = (1 + beta ** 2 * np.tan(np.pi * alpha / 2) ** 2) ** (1 / (2 * alpha))
        B_alpha_beta = (1 / alpha) * np.arctan(beta * np.tan(np.pi * alpha / 2))

    for i in range(n):
        U = np.random.uniform(-np.pi/2, np.pi/2)
        W = -np.log(1 - np.random.uniform(0,1))
        
        # Handle the case alpha = 1 separately
        if alpha != 1:
            part1 = np.sin(alpha * (U + B_alpha_beta)) / (np.cos(U) ** (1 / alpha))
            part2 = (np.cos(U - alpha * (U + B_alpha_beta)) / W) ** ((1 - alpha) / alpha)
            sample[i] = S_alpha_beta * part1 * part2
        else:
            sample[i] = (2 / np.pi) * ((np.pi / 2 + beta * U) * np.tan(U) - beta * np.log((np.pi / 2 * W * np.cos(U))/(np.pi+beta*U)))

    # Apply scaling and location shifting
    sample = gamma * sample + delta
    return sample

In [3]:
def char_fun_zolo(t, nu, etha, tau):
    """characteristic function to use from S2 to S5

    Parameters
    ----------------------------
    t : float
    the "instant" we want to check

    nu : float
    parameter of the alpha-stable distribution, >= 1/4.

    etha :float
    parameter of the alpha-stable distribution, np.sign(etha) <= min(1, 2*np.sqrt(nu)-1)

    tau : float
    parameter of the alpha-stable distribution, absolute value < inf
    ------------------------------
    """

    y = -np.exp((nu**(-1/2))*(np.log(np.abs(t)) + tau - 1j*(np.pi/2)*etha*np.sign(t)) + np.e*((nu**(-1/2))-1))
    return y
    

In [4]:
def emp_char_fun(sample, t):
    """function to evaluate the empirical characteristic of a function at an instant t

    Parameters
    -------------------------
    sample : array-like
    sample on which we want to evaluate the characteristic function

    t : int, float
    moment to estimate/generate
    --------------------------
    """
    expo_transfo = np.exp(-sample*1j*t)
    empirical = np.mean(expo_transfo)
    return empirical
    

$\theta = (\alpha, \beta, \gamma, \delta)$ 

N = 1000

$x^1,...,x^p$ ~ $\pi(x|\theta)$

$\pi_{LF}(\theta|y)$ = $\pi(\theta)\mathbb{E}_{\pi(x|\theta)}[K_{\epsilon} (y-x)]$ $\leftarrow$ we do monte carlo on this

$K_{\epsilon} (y-x)$ def by $S(y)$ ~ $N(S(x), \epsilon^2 \hat{\Sigma})$ $\leftarrow$ that would be our gaussian kernel, so we just draw $x_1,..., x_p$ ~ $\pi(x|\hat{\theta})$ then compute S(x), after that, estimate the cov(S(x)|$\hat{\theta}$) and then S(y) according to the previous distrib

$\hat{\Sigma}$ is an estimate of $Cov(S(x)\hat{\theta})$

$\Lambda$ = diag(0.25, 0.25, 1, 1)

Mutation kernel : $M_t(\theta_t) = \sum_{i=1}^{N} W_{t-1}^{(i)}(\theta_{t-1}^{(i)}) \phi(\theta_t ; \theta_{t-1}^{(i)})$

Gaussian kernel : $L(x,t) = \sum_{n = -M}^{M}f(x-n)G(n,t)$

with G(n,t) = $\frac{1}{\sqrt{2\pi t}}e^{-\frac{n^2}{2t}}$


As t increases, $\epsilon_t$ decreases

$\hat{c_t}$ is the 90th quantile of the weights

# For the rest we need to use : 

# $\alpha = 1.7$
# $\beta = 0.9$
# $\gamma = 10$
# $\delta = 10$

In [5]:
{}

{}

# Seed : 256652

In [6]:
np.random.seed(256652)

#creation des prior des parametres cf section 3.1

prior_alpha = np.random.uniform(1.1, 2., size=1000)
prior_beta = np.random.uniform(-1., 1, size=1000)
prior_gamma = np.random.uniform(0., 300., size=1000)
prior_delta = np.random.uniform(-300., 300., size=1000)

#scale parameters

#epsilon_t

mille = np.linspace(start=100, stop=1000, endpoint=True, num=10)[::-1]
cent = np.linspace(start=10, stop=100, endpoint=False, num=90)[::-1]
dix = np.linspace(start=5, stop=10, endpoint=False, num=10)[::-1]
cinq = np.linspace(start=3, stop=5, endpoint=False, num=40)[::-1]
trois = np.linspace(start=0, stop=3, endpoint=False, num=300)[::-1]

scale_param = np.concatenate((mille, cent, dix, cinq, trois))


N = 1000
varcov_lambda = np.array([0.25,0.,0.,0.,0.,0.25, 0., 0.,0.,0.,1.,0.,0.,0.,0.,1]).reshape([4,4])

In [7]:
def gaussian_ker(u=0, y=0):
    """gaussian kernel for weights
    
    Parameters
    -------------------
    y : float
    the point we have, the output

    u : float
    the point from which we want to calculate a weight
    ----------
    """

    w = (1/np.sqrt(2*np.pi))*np.exp(-(u-y)**2/2)
    return w

In [8]:
def mc_culloch_q(data, gamma=1):
    """fonction pour calculer les quantiles de mc_culloch utilises dans le papier

    Parameters
    --------------
    data : array
    les donnes pour lesquelles on veut calculer les quantiles

    gamma : float
    valeur 1 par defaut, d'apres le papier, on utilise le gamma utilise pour faire les simulations
    ----------------
    """
    data_sorted = np.sort(data)
    useful_quantiles = np.quantile(a=data_sorted, q=[0.95, 0.75, 0.5, 0.25, 0.05])
    q_95 = useful_quantiles[0]
    q_75 = useful_quantiles[1]
    q_50 = useful_quantiles[2]
    q_25 = useful_quantiles[3]
    q_05 = useful_quantiles[4]
    alpha_hat = (q_95-q_05)/(q_75-q_25)
    beta_hat = (q_95+q_05+2*q_50)/(q_95-q_05)
    gamma_hat = (q_75-q_25)/gamma
    #on a le gamma au denominateur parce que c'est comme ca qu on a genere nos donnees
    #d'apres le papier, ca va de prendre le gamma qu'on a utililse pour generer
    delta_hat = np.mean(data_simulation)
    S_1 = np.transpose(np.array((alpha_hat, beta_hat, gamma_hat, delta_hat)))
    return S_1

In [9]:
def zolotarev_transfo(sample, xi):
    """function to use for the estimation based on the zolotarev transformation

    Parameters
    --------------------------
    Sample : array-like
    Sample to do the transformation on

    xi : int, float
    The constant used in the transformation
    --------------------------
    """
    if xi<=0 or xi>1/2 :
        raise ValueError('Xi must be between 0 and 1/2')
    taille = len(sample)
    Z = []
    for i in range(int(taille/3)):
        transfo = sample[3*i-2] - xi*sample[3*i-1] - (1 - xi)*sample[3*i]
        Z.append(transfo)
    V = []
    U = []
    for i in range(len(Z)):
        V.append(np.log(np.abs(Z[i])))
        U.append(np.sign(sample[i]))
    V = np.array(V)
    U = np.array(U)
    S_U_squared = (np.std(U))**2
    S_V_squared = (np.std(V))**2
    nu_tilde = (6/(np.pi)**2)*S_V_squared - (3/2)*S_U_squared + 1
    etha_hat = np.mean(U)
    tau_hat = np.mean(V)
    nu_hat = 0
    if nu_tilde > ((1+np.abs(etha_hat))**2)/4:
        nu_hat = nu_tilde
    else:
        nu_hat = ((1+np.abs(etha_hat))**2)/4
    delta_hat = np.mean(sample)
    S_2 = np.array((nu_hat, etha_hat, tau_hat, delta_hat))
    return S_2

In [10]:
def u_hat(x, t):
    """function to use to calculate presss moments

    Parameters
    ---------------
    x : array-like

    the data

    t : int, float
    the real number we want to know the image by the function
    ---------------
    """

    y = np.arctan(np.sum(np.cos(t*x))/np.sum(np.sin(t*x)))
    return y

In [11]:
test_u = np.array((np.pi, np.pi, np.pi, np.pi))

In [12]:
u_hat(x=test_u,t=1/4)

0.7853981633974484

In [13]:
#fonction u_hat marche bien, on retombe sur arctan de 1

In [14]:
def presss_moments(sample, t_1=0.2, t_2=0.8, t_3=0.1, t_4=0.4):
    """function to calculate press's moments

    Parameters
    ----------------

    sample : array-like

    the data

    t_1 : int, float
    t_2 : int, float
    t_3 : int, float
    t_4 : int, float
    moments we want to evaluate, the default parameters are given by Koutrouvelis (1980)
    ---------------
    """
    
    #estimation of characteristic function at different moments
    transfo_1 = emp_char_fun(sample, t=t_1)
    transfo_2 = emp_char_fun(sample, t=t_2)
    transfo_3 = emp_char_fun(sample, t=t_3)
    transfo_4 = emp_char_fun(sample, t=t_4)
    log_gamma_hat_top = (np.log(np.abs(t_1))*np.log(-np.log(np.abs(transfo_2)))-np.log(np.abs(t_2))*np.log(-np.log(np.abs(transfo_1))))
    log_gamma_hat = log_gamma_hat_top/np.log(np.abs(t_1/t_2))
    alpha_hat = np.log(np.log(np.abs(transfo_1))/np.log(np.abs(transfo_2)))/np.log(np.abs(t_1/t_2))
    beta_hat_top = ((u_hat(x=sample, t=t_4)/t_4)-(u_hat(x=sample, t=t_3)/t_3))
    beta_hat_bot = ((np.abs(t_4)**(alpha_hat-1))-(np.abs(t_3)**(alpha_hat-1)))*((np.exp(log_gamma_hat))**alpha_hat)*np.tan((alpha_hat*np.pi)/2)
    beta_hat = beta_hat_top/beta_hat_bot
    delta_hat_top = (((np.abs(t_4))**(alpha_hat-1))*u_hat(x=sample, t=t_3)/t_3)-((((np.abs(t_3))**(alpha_hat-1))*u_hat(x=sample,t=t_4))/t_4)
    delta_hat_bot = (np.abs(t_4)**(alpha_hat-1)) - (np.abs(t_3)**(alpha_hat-1))
    delta_hat = delta_hat_top/delta_hat_bot
    y = np.array((alpha_hat, beta_hat, np.exp(log_gamma_hat), delta_hat))
    return y


    

In [15]:
test = generation(n=100000, alpha=2, beta=0, gamma=1, delta=0)
S_hat = zolotarev_transfo(test, xi=0.15)
print(S_hat)

[ 0.25130672 -0.00261003 -0.01364276  0.00217127]


In [16]:
#ca converge tres tres lentement

In [17]:
presss_moments(sample=test)

array([1.99924076e+00, 3.28992434e+04, 1.00025029e+00, 1.96382223e+01])

In [48]:
initial = np.zeros(4)
for i in range(250):
    test_2 = generation(n=1000, alpha=1.7, beta=0.9, gamma=10, delta=10)
    initial = np.vstack((initial, presss_moments(sample=test_2)))
    

resultat = np.zeros(4)
resultat[0] = np.mean(initial[:,0])
resultat[1] = np.mean(initial[:,1])
resultat[2] = np.mean(initial[:,2])
resultat[3] = np.mean(initial[:,3])
    

In [49]:
print(resultat)

[ 0.10663174  1.51575732  3.83288014 -4.61122643]


# Press
$\hat{\alpha}$ et $\hat{\delta}$ assez bons

Assez bof : $\hat{\gamma}$

Catastrophe : $\hat{\beta}$

# Le résultat précédent c'est quand on prenait un truc un peu au pif
# Quand on utilise leurs valeurs de $\alpha, \beta, \gamma, \delta$ en fait c'est pas mal
En fait c'est pas foufou...

In [20]:
np.zeros(4)


array([0., 0., 0., 0.])

In [21]:
%%capture
#data generation and definition of moments

data_simulation = generation(n=1000, alpha=2, beta=0.5, gamma=3, delta=4)

In [22]:
%%capture
#sort data to be sure the quantiles are well computed bc you never know
data_simulation = np.sort(data_simulation)

In [23]:
useful_quantiles = np.quantile(a=data_simulation, q=[0.95, 0.75, 0.5, 0.25, 0.05])

In [24]:
useful_quantiles

array([10.69440072,  6.4811886 ,  3.81208576,  1.11335256, -2.8903343 ])

In [25]:
q_95 = useful_quantiles[0]
q_75 = useful_quantiles[1]
q_50 = useful_quantiles[2]
q_25 = useful_quantiles[3]
q_05 = useful_quantiles[4]

### Mc Culloch's quantiles

In [26]:
alpha_hat = (q_95-q_05)/(q_75-q_25)
beta_hat = (q_95+q_05+2*q_50)/(q_95-q_05)
gamma_hat = (q_75-q_25)/3
#on a le 3 au denominateur parce que c'est comme ca qu on a genere nos donnees
#d'apres le papier, ca va de prendre le gamma qu'on a utililse pour generer
delta_hat = np.mean(data_simulation)

In [27]:
S_1 = np.transpose(np.array((alpha_hat, beta_hat, gamma_hat, delta_hat)))

In [28]:
S_1

array([2.53076564, 1.135704  , 1.78927868, 3.78614667])

In [29]:
delta_hat

3.786146667034114

Test runned with one million simulations, the estimators seemed to be biased.

def prior_lf_mc():
    "estimation monte carlo de la vraisemblance
    

def smc_prc_abc(N=1000, epsilon_t):
    """Algorithme ABC de l'appendix A de l'article.


    Parameters
    ---------------------------
    N : int
    Nombre d'echantillons crees pour les parametres

    epsilon_t : array
    Les marges d'acceptation pour l'algorithme
    """
    #les prior sont des uniformes cf partie 3.1
    prior_alpha = np.random.uniform(1.1, 2., size=N)
    prior_beta = np.random.uniform(-1., 1, size=N)
    prior_gamma = np.random.uniform(0., 300., size=N)
    prior_delta = np.random.uniform(-300., 300., size=N)
    prior_gen = np.vstack((prior_alpha,prior_beta,prior_delta, prior_gamma))
    
    #the weights
    

    

    

In [30]:
a = np.array((1,2,3,4,5))

In [31]:
a

array([1, 2, 3, 4, 5])

In [32]:
b = np.array((5,4,3,2,1))

In [33]:
a/b

array([0.2, 0.5, 1. , 2. , 5. ])

In [34]:
c = np.array((5,1,3,2,4))

In [35]:
test = np.vstack((a,b,c))

In [36]:
test

array([[1, 2, 3, 4, 5],
       [5, 4, 3, 2, 1],
       [5, 1, 3, 2, 4]])

In [37]:
test = np.hstack((a,b,c))

In [38]:
test

array([1, 2, 3, 4, 5, 5, 4, 3, 2, 1, 5, 1, 3, 2, 4])

In [39]:
np.linalg.norm(a - b)

6.324555320336759

In [40]:
test[:,0]

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
test[0,:]

In [41]:
def accept_reject_abc(N=1000, epsilon_t=0.5, alpha=2, beta=0, gamma=1, delta=0):
    """algorithme d'acceptation rejet 'basique', vu en cours

    Parameters
    ---------------------------------
    N : int
    nombre de simulations faites

    epsilon_t : array
    scale parameters, marge d'acceptation

    alpha, beta, gamma, delta :
    parametres pour les simulations
    -------------------------------
    """
    data = generation(n=N, alpha=alpha, beta=beta, gamma=gamma, delta=delta)
    #priors
    prior_alpha = np.random.uniform(1.1, 2., size=5*N)
    prior_beta = np.random.uniform(-1., 1, size=5*N)
    prior_gamma = np.random.uniform(0., 300., size=5*N)
    prior_delta = np.random.uniform(-300., 300., size=5*N)
    prior_gen = np.vstack((prior_alpha,prior_beta,prior_delta, prior_gamma))
    true_data_estim = zolotarev_transfo(sample=data, xi=0.15)
    bon_param = np.array(())
    alpha_test = 0
    beta_test = 0
    gamma_test = 0
    delta_test = 0
    for i in range(5*N):
        proposed_data = generation(n=N, alpha=prior_gen[:,i][0],
                                   beta=prior_gen[:,i][1],
                                   delta=prior_gen[:,i][2],
                                   gamma=prior_gen[:,i][3])
        alpha_test = prior_gen[:,i][0]
        beta_test = prior_gen[:,i][1]
        gamma_test = prior_gen[:,i][2]
        delta_test = prior_gen[:,i][3]
        estimated = mc_culloch_q(proposed_data, gamma=prior_gen[:,i][3])
        weights = gaussian_ker(estimated -true_data_estim)
        if np.linalg.norm(estimated-true_data_estim) < epsilon_t:
            return alpha_test, beta_test, gamma_test, delta_test
        else :
            continue

In [42]:
test_1 = accept_reject_abc(N=200, epsilon_t=0.3)

In [43]:
start = time.time()
test_1
end = time.time()
print(test_1, end - start)

None 4.1961669921875e-05


In [44]:
#avec une si petite distance ca ne marchait pas

In [51]:
start = time.time()
liste_parametres = []
for i in range(len(scale_param)):
    pouet = accept_reject_abc(N=200, epsilon_t=scale_param[-i])
    intermediaire = [scale_param[-i], pouet]
    liste_parametres.append(intermediaire)

end = time.time()

print(end - start)

802.6477806568146


In [52]:
#must change the parameters, N must be 200, too long otherwise

In [53]:
%%capture
liste_parametres

In [54]:
#not working very well, new distance needed

In [55]:
#Works probably, pb : wrong use of mc culloch quantiles, need to invert --> pb , how

In [56]:
liste_parametres

[[1000.0,
  (1.2376093000065582,
   0.5928972176503062,
   297.2045608500322,
   81.1764696080993)],
 [0.0, None],
 [0.01, None],
 [0.02, None],
 [0.03, None],
 [0.04, None],
 [0.05, None],
 [0.06, None],
 [0.07, None],
 [0.08, None],
 [0.09, None],
 [0.1, None],
 [0.11, None],
 [0.12, None],
 [0.13, None],
 [0.14, None],
 [0.15, None],
 [0.16, None],
 [0.17, None],
 [0.18, None],
 [0.19, None],
 [0.2, None],
 [0.21, None],
 [0.22, None],
 [0.23, None],
 [0.24, None],
 [0.25, None],
 [0.26, None],
 [0.27, None],
 [0.28, None],
 [0.29, None],
 [0.3, None],
 [0.31, None],
 [0.32, None],
 [0.33, None],
 [0.34, None],
 [0.35000000000000003, None],
 [0.36, None],
 [0.37, None],
 [0.38, None],
 [0.39, None],
 [0.4, None],
 [0.41000000000000003, None],
 [0.42, None],
 [0.43, None],
 [0.44, None],
 [0.45, None],
 [0.46, None],
 [0.47000000000000003, None],
 [0.48, None],
 [0.49, None],
 [0.5, None],
 [0.51, None],
 [0.52, None],
 [0.53, None],
 [0.54, None],
 [0.55, None],
 [0.56, None],
 [0.5

In [58]:
type(None)

NoneType

In [89]:
total = np.zeros(4)
for i in range(len(liste_parametres)):
    if type(liste_parametres[i][1]) != None:
        interm = np.array(liste_parametres[i][1])
        print(interm)
        total = np.vstack((total, interm))

[  1.2376093    0.59289722 297.20456085  81.17646961]
None


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 4 and the array at index 1 has size 1

In [68]:
if 1 == 2:
    print('no')
else:
    print('yes')

yes


In [73]:
liste_parametres[3][1]

In [80]:
for i in range(len(liste_parametres)):
    if liste_parametres[i][0] == 4.85:
        print(np.array(liste_parametres[i][1]))

[  1.63020396   0.90328829 -83.91701009 269.15386787]


In [90]:
#faudra changer aussi le format du test, pas liste mais passer direct en array